In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

class Imdb:
    
    url = 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc&start=1&ref_=adv_nxt'
    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html5lib')
    
    def __init__(self):
        self.all_movies = Imdb.soup.find_all(class_="lister-item mode-advanced")
        
        self.first = self.all_movies[0]

        self.fn = self.first.find(class_='lister-item-header').get_text().strip().replace('\n','').replace(' ' ,'')
        self.frt = self.first.find(class_='inline-block ratings-imdb-rating').get_text().strip().replace('\n','').replace(' ' ,'')
        self.fmeta = self.first.find(class_='inline-block ratings-metascore').get_text().strip('Metascore\n ')
        self.vg = self.first.find(class_='sort-num_votes-visible').get_text().strip().replace('\n','').replace(' ' ,'').split('|')

        self.vts = self.vg[0].split(':')[1].replace(',','')
        self.gs = self.vg[1].split(':')[1]
        
    def table(self):
        
        names,ratings,metas,votes,gross=[],[],[],[],[]

        for i in range (1,1001,50):
            url = f'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc&start={i}&ref_=adv_nxt'
            page = requests.get(url)
            soup = BeautifulSoup(page.content,'html5lib')

            all_movies = soup.find_all(class_="lister-item mode-advanced")

            for first in all_movies:
                #If metascore is given
                if first.find(class_='inline-block ratings-metascore'):
                    names.append(first.find(class_='lister-item-header').get_text().strip().replace('\n','').replace(' ' ,''))
                    ratings.append(first.find(class_='inline-block ratings-imdb-rating').get_text().strip().replace('\n','').replace(' ' ,''))
                    metas.append(first.find(class_='inline-block ratings-metascore').get_text().strip('Metascore\n '))
                    vg = first.find(class_='sort-num_votes-visible').get_text().strip().replace('\n','').replace(' ' ,'').split('|')

                    if len(vg)==2:
                        votes.append(vg[0].split(':')[1].replace(',',''))
                        gross.append(vg[1].split(':')[1])
                    else:
                        votes.append(vg[0].split(':')[1].replace(',',''))
                        gross.append(None)

                #If metascore is not given
                else:
                    names.append(first.find(class_="lister-item-header").get_text().replace(' ','').replace('\n',''))
                    ratings.append(first.find(class_="inline-block ratings-imdb-rating").get_text().strip())
                    metas.append(None)
                    vg=first.find(class_="sort-num_votes-visible").get_text().strip().replace(' ','').replace('\n','').split('|')
                    votes.append(vg[0].split(':')[1].replace(',',''))
                    gross.append(None)
                    
        self.names = names
        self.ratings = ratings
        self.metas = metas
        self.votes = votes
        self.gross = gross
            
                    
    def DataFrame(self):
        d={"Movie Name":self.names,'Ratings':self.ratings,'Metascore':self.metas,'Votes':self.votes,'Gross':self.gross}
        df=pd.DataFrame(d)
        return df.head(100)


In [15]:
imdb = Imdb()
imdb.table()

In [16]:
imdb.DataFrame()

,Movie Name,Ratings,Metascore,Votes,Gross
0,1.Logan(2017),8.1,77,729077,$226.28M
1,2.Thor:Ragnarok(2017),7.9,74,697578,$315.06M
2,3.GuardiansoftheGalaxyVol.2(2017),7.6,67,649764,$389.81M
3,4.WonderWoman(2017),7.4,76,637425,$412.56M
4,5.Dunkirk(2017),7.8,94,632058,$188.37M
...,...,...,...,...,...
95,96.TheGoodDoctor(2017–),8.1,None,88532,None
96,97.GameofThrones(2011–2019)Episode:TheSpoilsof...,9.7,None,86774,None
97,98.AmericanGods(2017–2021),7.7,None,84316,None
98,99.GoinginStyle(2017),6.6,50,84310,$45.02M
